In [6]:
!pip install pandas
!pip install numpy
!pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/58/d8/f3249232fa79a70cb40595282813e61453c1e76da3e1a44b77a63dd8d0cb/scikit_learn-1.7.0-cp312-cp312-macosx_12_0_arm64.whl.metadata
  Using cached scikit_learn-1.7.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (31 kB)
  Obtaining dependency information for scipy>=1.8.0 from https://files.pythonhosted.org/packages/07/9c/0ddb0d0abdabe0d181c1793db51f02cd59e4901da6f9f7848e1f96759f0d/scipy-1.15.3-cp312-cp312-macosx_14_0_arm64.whl.metadata
  Using cached scipy-1.15.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (61 kB)
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/7d/4f/1195bbac8e0c2acc5f740661631d8d750dc38d4a32b23ee5df3cde6f4e0d

In [2]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

def fill_na_neighbors(df_subset, degree=2):
    df_filled = df_subset.copy()
    years = np.array([int(col) for col in df_filled.columns])
    dtypes = {col: df_filled[col].dtype for col in df_filled.columns}

    for idx, row in df_filled.iterrows():
        vals = row.values
        mask = ~pd.isna(vals)

        if mask.sum() == 0:
            continue
        is_integer = np.issubdtype(row.dtype, np.integer)
        X_known = years[mask].reshape(-1, 1)
        y_known = vals[mask].astype(float)

        if mask.sum() > degree + 1:
            model = make_pipeline(
                PolynomialFeatures(degree),
                LinearRegression()
            )
        else:
            model = LinearRegression()

        model.fit(X_known, y_known)
        preds = model.predict(years.reshape(-1, 1))

        if not is_integer:
            new_vals = np.where(~mask, preds, vals)
        else:
            new_vals = np.where(~mask, np.round(preds).astype(int), vals)
            new_vals = np.clip(new_vals, a_min=0, a_max=None)

        for i, col in enumerate(df_filled.columns):
            df_filled.at[idx, col] = dtypes[col].type(new_vals[i])

    return df_filled

def clean_year_col(col_name):
    if '[' in col_name and 'YR' in col_name:
        return col_name.split(' ')[0].strip()
    return col_name


file_path = "/Users/ekaterinamakhotkina/PycharmProjects/pythonProject8/f4dc79c5-2efc-4406-957a-e8dc7fc523fc_Series - Metadata.csv"
df = pd.read_csv(file_path, encoding='latin1', on_bad_lines='warn')

df = df.drop_duplicates()

str_cols = df.select_dtypes(include='object').columns
df[str_cols] = df[str_cols].apply(lambda col: col.str.strip())

df = df.loc[:, df.isnull().mean() < 0.5]

for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].fillna('')

for col in df.columns:
    if np.issubdtype(df[col].dtype, np.number):
        df[col] = df[col].fillna(df[col].median())

df.columns = [clean_year_col(col) for col in df.columns]


year_cols = [col for col in df.columns if col.isdigit()]

invalid_values = ['..', '-', 'N/A', 'NA', 'n/a', 'na', '']
for col in year_cols:
    df[col] = df[col].replace(invalid_values, np.nan)

df[year_cols] = df[year_cols].apply(pd.to_numeric, errors='coerce')

if year_cols:
    df_years_filled = fill_na_neighbors(df[year_cols])
    df[year_cols] = df_years_filled
    
df = df.dropna()

str_cols = df.select_dtypes(include='object').columns
df = df.loc[~(df[str_cols] == '').all(axis=1)]


df.to_csv("cleaned_metadata_2.csv", index=False)

/var/folders/7r/pnhwy0957_b569dd57vr9nqw0000gn/T/ipykernel_6740/2285547660.py:51: ParserWarning: Skipping line 1710: expected 29 fields, saw 30
Skipping line 1861: expected 29 fields, saw 30
Skipping line 1862: expected 29 fields, saw 30
Skipping line 2069: expected 29 fields, saw 30
Skipping line 2320: expected 29 fields, saw 35
Skipping line 2333: expected 29 fields, saw 31
Skipping line 2334: expected 29 fields, saw 30
Skipping line 2771: expected 29 fields, saw 32
Skipping line 2772: expected 29 fields, saw 32
Skipping line 2773: expected 29 fields, saw 32
Skipping line 2774: expected 29 fields, saw 32
Skipping line 2775: expected 29 fields, saw 32

  df = pd.read_csv(file_path, encoding='latin1', on_bad_lines='warn')


In [4]:
df_cleaned=pd.read_csv('/Users/ekaterinamakhotkina/PycharmProjects/pythonProject8/cleaned_metadata_1.csv')
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262 entries, 0 to 1261
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  1262 non-null   object 
 1   Country Code  1262 non-null   object 
 2   Series Name   1262 non-null   object 
 3   Series Code   1218 non-null   object 
 4   2000          1262 non-null   float64
 5   2001          1262 non-null   float64
 6   2002          1262 non-null   float64
 7   2003          1262 non-null   float64
 8   2004          1262 non-null   float64
 9   2005          1262 non-null   float64
 10  2006          1262 non-null   float64
 11  2008          1262 non-null   float64
 12  2009          1262 non-null   float64
 13  2010          1262 non-null   float64
 14  2011          1262 non-null   float64
 15  2012          1262 non-null   float64
 16  2013          1262 non-null   float64
 17  2014          1262 non-null   float64
 18  2015          1262 non-null 

In [3]:
df_cleaned=pd.read_csv('/Users/ekaterinamakhotkina/PycharmProjects/pythonProject8/cleaned_metadata_2.csv')
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262 entries, 0 to 1261
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  1262 non-null   object 
 1   Country Code  1262 non-null   object 
 2   Series Name   1262 non-null   object 
 3   Series Code   1218 non-null   object 
 4   2000          1262 non-null   float64
 5   2001          1262 non-null   float64
 6   2002          1262 non-null   float64
 7   2003          1262 non-null   float64
 8   2004          1262 non-null   float64
 9   2005          1262 non-null   float64
 10  2006          1262 non-null   float64
 11  2008          1262 non-null   float64
 12  2009          1262 non-null   float64
 13  2010          1262 non-null   float64
 14  2011          1262 non-null   float64
 15  2012          1262 non-null   float64
 16  2013          1262 non-null   float64
 17  2014          1262 non-null   float64
 18  2015          1262 non-null 

In [7]:
df_cleaned=pd.read_csv('/Users/ekaterinamakhotkina/PycharmProjects/pythonProject8/cleaned_metadata_2.csv')
df_cleaned

,Country Name,Country Code,Series Name,Series Code,2000,2001,2002,2003,2004,2005,...,2009,2010,2011,2012,2013,2014,2015,2017,2018,2019
0,Russian Federation,RUS,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,100.0,99.9,99.9,99.9,99.9,99.8,...,99.7,99.6,99.6,99.5,99.5,99.6,99.5,99.5,99.5,99.5
1,Russian Federation,RUS,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,99.9,99.8,99.8,99.7,99.7,99.6,...,99.3,99.3,99.2,99.1,99.1,99.1,99.1,99.0,99.0,99.0
2,Russian Federation,RUS,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,99.9,99.9,99.8,99.8,99.7,99.7,...,99.4,99.4,99.3,99.3,99.3,99.2,99.3,99.2,99.2,99.2
3,Russian Federation,RUS,Access to electricity (% of population),EG.ELC.ACCS.ZS,97.8,97.9,97.9,97.9,98.0,98.0,...,98.4,99.3,99.2,100.0,100.0,100.0,95.5,100.0,100.0,99.8
4,Russian Federation,RUS,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,96.1,96.4,96.6,96.8,97.0,97.2,...,98.0,100.0,99.6,100.0,100.0,100.0,96.6,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1257,NV.SRV.TOTL.KD,CC BY-4.0,"Services, value added (constant 2015 US$)",NaN,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,...,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0
1258,NV.SRV.EMPL.KD,CC BY-4.0,"Services, value added per worker (constant 201...",NaN,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,...,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0
1259,EN.GHG.TOT.ZG.AR5,CC BY-4.0,Total greenhouse gas emissions excluding LULUC...,NaN,1990.0,1990.0,1990.0,1990.0,1990.0,1990.0,...,1990.0,1990.0,1990.0,1990.0,1990.0,1990.0,1990.0,1990.0,1990.0,1990.0
1260,ER.GDP.FWTL.M3.KD,CC BY-4.0,"Water productivity, total (constant 2015 US$ G...",NaN,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,...,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0,2015.0
